In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import re

from selenium import webdriver

In [2]:
url = 'https://www.buyrentkenya.com/flats-apartments-for-rent/nairobi'

In [3]:
# set the url we want to visit
res = requests.get(url)
res.status_code

200

In [4]:
soup = BeautifulSoup(res.content,'lxml')

In [5]:
# get id
#for i in soup.find_all('i', {'class': 'fa fa-heart listing-favourite'})[:2]:
   #print(i.attrs['data-t-listing_id'])

# get prices
#for i in soup.find_all('a', {'class': 'item-price'})[:2]:
     #print(int(i.text.split()[1].replace(',','')))

# get bedrooms
#for i in soup.find_all('span', {'class': 'h-beds'})[:2]:
    #print(i.text)

# get location
#for i in soup.find_all('div', {'class': 'property-location'})[:2]:
    #print(i.text)

# get listing type
#for i in soup.find_all('i', {'class': 'fa fa-heart listing-favourite'})[:2]:
    #print(i.attrs['data-t-listing_type'])

# get listing category
#for i in soup.find_all('i', {'class': 'fa fa-heart listing-favourite'})[:2]:
    #print(i.attrs['data-t-listing_category'])

In [6]:
#sleep(5)
#for row in soup.find_all('div', {'class': 'result-card-item mt-2 mb-3'}):
    #id = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_id']
    #print(id)
    
    #category = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_category']
    #try:
        #print(category.text)
    #except:
        #print(None)
        
    #list_type = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_type']
    #try:
        #print(list_type.text)
    #except:
        #print(None)
        
    #beds = row.find('span', {'class': 'h-beds'})
    #try:
        #print(beds.text)
    #except:
        #print(None)
    
    #baths = row.find('span', {'class': 'h-baths'})
    #try:
        #print(baths.text)
    #except:
        #print(None)
    
    #price = row.find('a', {'class': 'item-price'})
    #try:
        #print(price.text.split()[1].replace(',',''))
    #except:
        #print(None)

    #sq_ft_price = row.find('a', {'class': 'item-sub-price'})
    #try:
        #print(sq_ft_price.text.split()[1].replace(',',''))
    #except:
        #print(None)    
        
    #loc = row.find('div', {'class': 'property-location'}).text.split(',')
    # Adapted from Matt's code to separate into district, neighborhood, and street
    #print(loc[-1].strip())
    #try:
        #print(loc[-2].strip())
    #except:
        #print(None)
    #try:
        #print(loc[-3].strip())
    #except:
        #print(None)
        
    #print()

In [7]:
def apt_grabber(soup, df):
    for row in soup.find_all('div', {'class': 'result-card-item mt-2 mb-3'}):
        id = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_id']
    
        category = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_category']
        try:
            category = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_category']
        except:
            category = 'None'
        
        list_type = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_type']
        try:
            list_type = row.find('i', {'class': 'fa fa-heart listing-favourite'}).attrs['data-t-listing_type']
        except:
            list_type = 'None'
        
        beds = row.find('span', {'class':'h-beds'})
        try:
            beds = row.find('span', {'class':'h-beds'}).text
        except:
            beds = 'None'
        
        baths = row.find('span', {'class':'h-baths'})
        try:
            baths = row.find('span', {'class':'h-baths'}).text
        except:
            baths = 'None'
        
        sq_ft_price = row.find('a', {'class': 'item-sub-price'})
        try:
            sq_ft_price = row.find('a', {'class': 'item-sub-price'}).text.split()[1].replace(',','')
        except:
            sq_ft_price = 'None'
        
        price = row.find('a', {'class': 'item-price'})
        try:
            price = row.find('a', {'class': 'item-price'}).text.split()[1].replace(',','')
        except:
            price = 'None'
        
        loc = row.find('div', {'class': 'property-location'}).text.split(',')
        
        # Adapted from Matt's code to separate into district, neighborhood, and street
        district = loc[-1].strip()
        try:
            neighborhood = loc[-2].strip()
        except:
            neighborhood = 'None'
        try:
            street = loc[-3].strip()
        except:
            street = 'None'
        
        df.loc[len(df)] = [id, category, list_type, beds, baths, sq_ft_price_KES, price, district, neighborhood, street]

In [8]:
df_rent = pd.DataFrame(columns=['id', 'category', 'list_type', 'beds', 'baths', 'sq_ft_price_KES', 'price_KES', 'district', 'neighborhood', 'street'])

url = 'https://www.buyrentkenya.com/flats-apartments-for-rent/nairobi' #url for flats & apartments for rent
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)

for i in range(104):
    sleep(5)
    soup_object = BeautifulSoup(driver.page_source, 'lxml')
    apt_grabber(soup_object, df_rent)
    next_button = driver.find_element_by_link_text('Next')
    next_button.click()

driver.close()

In [9]:
df_rent.head()

,id,category,list_type,beds,baths,sq_ft_price,price_KES,district,neighborhood,street
0,3394167,Flat & Apartment,For Rent,2,None,None,120000,Westlands,Brookside,None
1,3399035,Flat & Apartment,For Rent,2,None,77,96000,Ridgeways,None,None
2,3433894,Flat & Apartment,For Rent,2,None,46,70000,Dagoretti North,Valley Arcade,Lavington
3,3433821,Flat & Apartment,For Rent,2,2,None,200000,Westlands,Westlands Area,Kingfisher Nest
4,3433777,Flat & Apartment,For Rent,4,None,None,110000,Westlands,Parklands,1St Parklands


In [10]:
df_rent.shape

(2080, 10)

In [11]:
df_rent.to_csv('nairobi_rent.csv', index=False)

In [12]:
df_sale = pd.DataFrame(columns=['id', 'category', 'list_type', 'beds', 'baths', 'sq_ft_price', 'price_KES', 'district', 'neighborhood', 'street'])

url = 'https://www.buyrentkenya.com/flats-apartments-for-sale/nairobi' #url for flats & apartments for sale
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)

for i in range(96):
    sleep(5)
    soup_object = BeautifulSoup(driver.page_source, 'lxml')
    apt_grabber(soup_object, df_sale)
    next_button = driver.find_element_by_link_text('Next')
    next_button.click()

driver.close()

In [13]:
df_sale.head()

,id,category,list_type,beds,baths,sq_ft_price,price_KES,district,neighborhood,street
0,3428432,Flat & Apartment,For Sale,3,2,None,18000000,Dagoretti North,Kileleshwa,Hamisi Road
1,3433663,Studio,For Sale,1,1,None,2000000,Dagoretti North,Naivasha Road,None
2,3433810,Flat & Apartment,For Sale,3,4,7548,13000000,Dagoretti North,Kilimani,Kilungu Road
3,3433808,Studio,For Sale,1,1,10839,3500000,Dagoretti North,Kilimani,Dennis Pritt
4,3433877,Flat & Apartment,For Sale,3,3,None,15500000,Westlands,Parklands,None


In [14]:
df_sale.shape

(1920, 10)

In [64]:
df_nairobi = pd.concat([df_rent, df_sale])
df_nairobi

,id,category,list_type,beds,baths,sq_ft_price,price_KES,district,neighborhood,street
0,3394167,Flat & Apartment,For Rent,2,None,None,120000,Westlands,Brookside,None
1,3399035,Flat & Apartment,For Rent,2,None,77,96000,Ridgeways,None,None
2,3433894,Flat & Apartment,For Rent,2,None,46,70000,Dagoretti North,Valley Arcade,Lavington
3,3433821,Flat & Apartment,For Rent,2,2,None,200000,Westlands,Westlands Area,Kingfisher Nest
4,3433777,Flat & Apartment,For Rent,4,None,None,110000,Westlands,Parklands,1St Parklands
...,...,...,...,...,...,...,...,...,...,...
1915,3400085,Flat & Apartment,For Sale,3,4,11476,18900000,Dagoretti North,Kileleshwa,Kileleshwa
1916,3403249,Flat & Apartment,For Sale,3,None,None,19000000,Dagoretti North,Kilimani,None
1917,3400575,Studio,For Sale,1,1,8361,3600000,Dagoretti North,Kileleshwa,Kileleshwa
1918,3386205,Flat & Apartment,For Sale,2,2,None,7950000,Ngong Road,None,None


In [65]:
df_nairobi.shape

(4000, 10)

In [66]:
df_nairobi.to_csv('nairobi_all_listings.csv', index=False)